In [52]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [5]:
options = webdriver.ChromeOptions()
driver_path = '/Users/Jenny/Desktop/공모전/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

/var/folders/x_/s6rt1hw118zdhq387czgy5q40000gn/T/ipykernel_5730/3616131504.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


In [45]:
url = 'https://e-childschoolinfo.moe.go.kr/kinderMt/combineFind.do'
driver.get(url)

In [46]:
# 유치원 카테고리 버튼 클릭
kin_button = driver.find_element(By.XPATH, '//*[@id="filterArea"]/div[2]/ul/li[2]/a/span')
kin_button.click()

search1_button = driver.find_element(By.XPATH, '//*[@id="schArea"]/div[2]/a[1]')
search1_button.click()

In [47]:
# 휴원, 폐원 제외 버튼 클릭
rest_except = driver.find_element(By.XPATH, '//*[@id="optionType2"]/div[4]/div/ul/li[1]/div/label')
rest_except.click()

search1_button = driver.find_element(By.XPATH, '//*[@id="schArea"]/div[2]/a[1]')
search1_button.click()

close_except = driver.find_element(By.XPATH, '//*[@id="optionType2"]/div[4]/div/ul/li[2]/div/label')
close_except.click()

search1_button = driver.find_element(By.XPATH, '//*[@id="schArea"]/div[2]/a[1]')
search1_button.click()

In [48]:
# 서울 특별시 지역 설정
drop_box = driver.find_element(By.XPATH, '//*[@id="layerSelect-local"]/label')
drop_box.click()

seoul_button = driver.find_element(By.XPATH, '//*[@id="list-local-1"]/li[2]/a')
seoul_button.click()

set_button = driver.find_element(By.XPATH, '/html/body/div[10]/div[2]/a[2]')
set_button.click()

search1_button = driver.find_element(By.XPATH, '//*[@id="schArea"]/div[2]/a[1]')
search1_button.click()

list_button = driver.find_element(By.XPATH, '//*[@id="listCnt_displayAtag"]')
list_button.click()

list_50 = driver.find_element(By.XPATH, '//*[@id="listCnt_aTag_50"]')
list_50.click()

In [50]:
kin_list = []

total_pages = 16

for page in range(1, total_pages):

    interval = 1 # 1초에 한번씩 스크롤 내림

    # 현재 문서 높이를 가져와서 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height


    columns = ['kin_type', 'kin_name', 'kin_addr'] # 사립/공립 형태, 유치원 이름, 주소
    values = [] # 유치원 정보를 담을 빈 리스트 생성

    # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
    soup = BeautifulSoup(driver.page_source, 'lxml')
    parent_element = soup.find('div', class_='lists')
    ul_element = parent_element.find('ul')
    data_rows = ul_element.find_all('li')

    for i, row in enumerate(data_rows):

        print('{}번째 크롤링'.format(i+1))
        blank = []
    
        # 유치원 형태 크롤링
        kin_type = row.find('span', {'class' : "est type1"})
        if kin_type:
            kin_type = kin_type.get_text().strip()
            blank.append(kin_type)
        else:
            blank.append('Something is wrong')
            print('{}번째 유치원 형태 가져올 때 문제 발생'.format(i+1))
            continue

        #유치원 이름 크롤링
        kin_name = row.find('a', {'class' : "underline"})
        if kin_name:
            kin_name = kin_name.get_text().strip()
            blank.append(kin_name)
        else:
            blank.append('Something is wrong')
            print('{}번째 유치원 이름 가져올 때 문제 발생'.format(i+1))
            continue

        #유치원 주소 크롤링
        kin_addr = row.find_all('span')[6]
        if kin_addr:
            kin_name = kin_addr.get_text().strip()
            blank.append(kin_addr)
        else:
            blank.append('Something is wrong')
            print('{}번째 유치원 주소 가져올 때 문제 발생'.format(i+1))
            continue  

        values.append(blank)
        print('---------------------------------------------------') 


    if page > 5:
        if page < 11:
            next_page_button_xpath = '//*[@id="resultArea"]/div[3]/div[3]/a[' + str(page - 2) + ']'
            next_page_button = driver.find_element(By.XPATH, next_page_button_xpath)
            next_page_button.click()

        elif page < 16:
            next_page_button_xpath = '//*[@id="resultArea"]/div[3]/div[3]/a[' + str(page - 7) + ']'
            next_page_button = driver.find_element(By.XPATH, next_page_button_xpath)
            next_page_button.click()

        #elif page < 17:
            #next_page_button_xpath = '//*[@id="resultArea"]/div[3]/div[3]/a[' + str(page - 12) + ']'
            #next_page_button = driver.find_element(By.XPATH, next_page_button_xpath)
            #next_page_button.click()

        elif page == 16:
            break

    else:
        next_page_button_xpath = '//*[@id="resultArea"]/div[3]/div[3]/a[' + str(page + 3) + ']'
        next_page_button = driver.find_element(By.XPATH, next_page_button_xpath)
        next_page_button.click()


kin_list = pd.DataFrame(values, columns = columns)

# page 2: //*[@id="resultArea"]/div[3]/div[3]/a[4]
# page 6: //*[@id="resultArea"]/div[3]/div[3]/a[8]
# page 7: //*[@id="resultArea"]/div[3]/div[3]/a[4]
# page 11: //*[@id="resultArea"]/div[3]/div[3]/a[8]

1번째 크롤링
---------------------------------------------------
2번째 크롤링
---------------------------------------------------
3번째 크롤링
---------------------------------------------------
4번째 크롤링
---------------------------------------------------
5번째 크롤링
---------------------------------------------------
6번째 크롤링
---------------------------------------------------
7번째 크롤링
---------------------------------------------------
8번째 크롤링
---------------------------------------------------
9번째 크롤링
---------------------------------------------------
10번째 크롤링
---------------------------------------------------
11번째 크롤링
---------------------------------------------------
12번째 크롤링
---------------------------------------------------
13번째 크롤링
---------------------------------------------------
14번째 크롤링
---------------------------------------------------
15번째 크롤링
---------------------------------------------------
16번째 크롤링
---------------------------------------------------
17번째 크롤링
------------------------

In [51]:
kin_list

,kin_type,kin_name,kin_addr
0,사립(사인),푸른동산유치원,[서울특별시 구로구 서해안로 2201]
1,사립(사인),푸른솔유치원,[서울특별시 동대문구 전농로10길 20]
2,사립(사인),푸른유치원,[서울특별시 강남구 일원로14길 25]
3,사립(사인),풍천유치원,[서울특별시 노원구 노원로 522]
4,사립(사인),프라임유치원,[서울특별시 광진구 광나루로56길 5]
5,사립(사인),하나유치원,[서울특별시 강북구 솔샘로 174]
6,사립(사인),하나유치원,[서울특별시 서초구 잠원로8길 13]
7,사립(법인),하늘유치원,[서울특별시 관악구 양산길 18]
8,사립(사인),하늘유치원,[서울특별시 노원구 초안산로 89]
9,사립(사인),학여울유치원,[서울특별시 노원구 동일로207길 186]
